In [5]:
import requests
import sys
import json
from dotenv import load_dotenv
import os
import re
import pandas as pd
from functools import wraps
import time
from sshtunnel import SSHTunnelForwarder
import psycopg2
import hashlib
import undetected_chromedriver as uc
from webdriver_manager.chrome import ChromeDriverManager
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
load_dotenv()
api_key=os.getenv('API_KEY')

response_stock_list=requests.get(f'https://fmpcloud.io/api/v3/stock/list?apikey={api_key}')
data_stock_list=response_stock_list.json()

required_companies_symbol=[]
for company in data_stock_list:
    required_companies_symbol.append(company['symbol'])

def send_mail(send_from: str,subject: str,text: str,send_to: list,files=None):
    send_to = default_address if not send_to else send_to
    msg = MIMEMultipart()
    username = 'faghost6201@gmail.com'
    password = 'mochdbbyiiwmxzzg'
    msg['From'] = send_from
    msg['To'] = ', '.join(send_to)
    msg['Subject'] = subject
    part1 = MIMEText(text, 'plain')
    msg.attach(part1)
    for f in files or []:
        with open(f, "rb") as fil:
            ext = f.split('.')[-1:]
            attachedfile = MIMEApplication(fil.read(), _subtype=ext)
            attachedfile.add_header('content-disposition',
                                    'attachment',
                                    filename=basename(f))
        msg.attach(attachedfile)

    if '@gmail' in send_from:
        smtp = smtplib.SMTP(host="smtp.gmail.com", port=587)
    elif '@yahoo' in send_from:
        smtp = smtplib.SMTP(host="smtp.mail.yahoo.com", port=587)
    elif '@outlook' in send_from or '@hotmail' in send_from:
        smtp = smtplib.SMTP(host="smtp-mail.outlook.com", port=587)
    elif '@verizonwireless' in send_from:
        smtp = smtplib.SMTP(host="smtp.verizon.net", port=465)

    smtp.starttls()
    smtp.login(username, password)
    smtp.sendmail(send_from, send_to, msg.as_string())
    smtp.close()

def replace_values(list_to_replace, item_to_replace, item_to_replace_with):
    return [item_to_replace_with if item == item_to_replace else item for item in list_to_replace]

DB_ENV_PROD=1


if DB_ENV_PROD==0:
    database=os.getenv('LOCAL_DB_NAME')
    user=os.getenv('LOCAL_DB_USER')
    password=os.getenv('LOCAL_DB_PASSWORD')
    port=int(os.getenv('DB_PORT'))
    host=os.getenv('DB_HOST')
    
    def database_connection(host=host,database=database,user=user,password=password,port=port):
        conn = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password,
        port=port)
        return conn

    make_table_query='''
    CREATE TABLE IF NOT EXISTS financial_ratios_ttm (
            hashed_key text NOT NULL PRIMARY KEY,
            symbol VARCHAR(255),
            asset_turnover_ttm float(24),
            capital_expenditure_coverage_ratio_ttm float(24),
            cash_conversion_cycle_ttm float(24),
            cash_flow_coverage_ratios_ttm float(24),
            cash_flow_to_debt_ratio_ttm float(24),
            cash_per_share_ttm float(24),
            cash_ratio_ttm float(24),
            company_equity_multiplier_ttm float(24),
            current_ratio_ttm float(24),
            days_of_inventory_outstanding_ttm float(24),
            days_of_payables_outstanding_ttm float(24),
            days_of_sales_outstanding_ttm float(24),
            debt_equity_ratio_ttm float(24),
            debt_ratio_ttm float(24),
            dividend_paid_and_capex_coverage_ratio_ttm float(24),
            dividend_per_share_ttm float(24),
            dividend_yiel_percentage_ttm float(24),
            dividend_yiel_ttm float(24),
            dividend_yield_ttm float(24),
            ebit_per_revenue_ttm float(24),
            ebt_per_ebit_ttm float(24),
            effective_tax_rate_ttm float(24),
            enterprise_value_multiple_ttm float(24),
            fixed_asset_turnover_ttm float(24),
            free_cash_flow_operating_cash_flow_ratio_ttm float(24),
            free_cash_flow_per_share_ttm float(24),
            gross_profit_margin_ttm float(24),
            interest_coverage_ttm float(24),
            inventory_turnover_ttm float(24),
            long_term_debt_to_capitalization_ttm float(24),
            net_income_per_ebtttm float(24),
            net_profit_margin_ttm float(24),
            operating_cash_flow_per_share_ttm float(24),
            operating_cash_flow_sales_ratio_ttm float(24),
            operating_cycle_ttm float(24),
            operating_profit_margin_ttm float(24),
            payables_turnover_ttm float(24),
            payout_ratio_ttm float(24),
            pe_ratio_ttm float(24),
            peg_ratio_ttm float(24),
            pretax_profit_margin_ttm float(24),
            price_book_value_ratio_ttm float(24),
            price_cash_flow_ratio_ttm float(24),
            price_earnings_ratio_ttm float(24),
            price_earnings_to_growth_ratio_ttm float(24),
            price_fair_value_ttm float(24),
            price_sales_ratio_ttm float(24),
            price_to_book_ratio_ttm float(24),
            price_to_free_cash_flows_ratio_ttm float(24),
            price_to_operating_cash_flows_ratio_ttm float(24),
            price_to_sales_ratio_ttm float(24),
            quick_ratio_ttm float(24),
            receivables_turnover_ttm float(24),
            return_on_assets_ttm float(24),
            return_on_capital_employed_ttm float(24),
            return_on_equity_ttm float(24),
            short_term_coverage_ratios_ttm float(24),
            total_debt_to_capitalization_ttm float(24)
            );
    '''
    conn=database_connection()
    cur = conn.cursor()
    cur.execute(make_table_query)
    conn.commit()

    options = uc.ChromeOptions()
    options.headless=True
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')

    driver = uc.Chrome(executable_path=ChromeDriverManager().install(),use_subprocess=True,options=options)
    driver.get('https://fmpcloud.io/api/v3')
    driver.implicitly_wait(30)
    
    insert_sql = '''
    INSERT INTO financial_ratios_ttm (hashed_key,symbol,asset_turnover_ttm,capital_expenditure_coverage_ratio_ttm,cash_conversion_cycle_ttm,cash_flow_coverage_ratios_ttm,cash_flow_to_debt_ratio_ttm,cash_per_share_ttm,cash_ratio_ttm,company_equity_multiplier_ttm,current_ratio_ttm,days_of_inventory_outstanding_ttm,days_of_payables_outstanding_ttm,days_of_sales_outstanding_ttm,debt_equity_ratio_ttm,debt_ratio_ttm,dividend_paid_and_capex_coverage_ratio_ttm,dividend_per_share_ttm,dividend_yiel_percentage_ttm,dividend_yiel_ttm,dividend_yield_ttm,ebit_per_revenue_ttm,ebt_per_ebit_ttm,effective_tax_rate_ttm,enterprise_value_multiple_ttm,fixed_asset_turnover_ttm,free_cash_flow_operating_cash_flow_ratio_ttm,free_cash_flow_per_share_ttm,gross_profit_margin_ttm,interest_coverage_ttm,inventory_turnover_ttm,long_term_debt_to_capitalization_ttm,net_income_per_ebtttm,net_profit_margin_ttm,operating_cash_flow_per_share_ttm,operating_cash_flow_sales_ratio_ttm,operating_cycle_ttm,operating_profit_margin_ttm,payables_turnover_ttm,payout_ratio_ttm,pe_ratio_ttm,peg_ratio_ttm,pretax_profit_margin_ttm,price_book_value_ratio_ttm,price_cash_flow_ratio_ttm,price_earnings_ratio_ttm,price_earnings_to_growth_ratio_ttm,price_fair_value_ttm,price_sales_ratio_ttm,price_to_book_ratio_ttm,price_to_free_cash_flows_ratio_ttm,price_to_operating_cash_flows_ratio_ttm,price_to_sales_ratio_ttm,quick_ratio_ttm,receivables_turnover_ttm,return_on_assets_ttm,return_on_capital_employed_ttm,return_on_equity_ttm,short_term_coverage_ratios_ttm,total_debt_to_capitalization_ttm)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    ON CONFLICT (hashed_key) DO UPDATE SET
    (symbol,asset_turnover_ttm,capital_expenditure_coverage_ratio_ttm,cash_conversion_cycle_ttm,cash_flow_coverage_ratios_ttm,cash_flow_to_debt_ratio_ttm,cash_per_share_ttm,cash_ratio_ttm,company_equity_multiplier_ttm,current_ratio_ttm,days_of_inventory_outstanding_ttm,days_of_payables_outstanding_ttm,days_of_sales_outstanding_ttm,debt_equity_ratio_ttm,debt_ratio_ttm,dividend_paid_and_capex_coverage_ratio_ttm,dividend_per_share_ttm,dividend_yiel_percentage_ttm,dividend_yiel_ttm,dividend_yield_ttm,ebit_per_revenue_ttm,ebt_per_ebit_ttm,effective_tax_rate_ttm,enterprise_value_multiple_ttm,fixed_asset_turnover_ttm,free_cash_flow_operating_cash_flow_ratio_ttm,free_cash_flow_per_share_ttm,gross_profit_margin_ttm,interest_coverage_ttm,inventory_turnover_ttm,long_term_debt_to_capitalization_ttm,net_income_per_ebtttm,net_profit_margin_ttm,operating_cash_flow_per_share_ttm,operating_cash_flow_sales_ratio_ttm,operating_cycle_ttm,operating_profit_margin_ttm,payables_turnover_ttm,payout_ratio_ttm,pe_ratio_ttm,peg_ratio_ttm,pretax_profit_margin_ttm,price_book_value_ratio_ttm,price_cash_flow_ratio_ttm,price_earnings_ratio_ttm,price_earnings_to_growth_ratio_ttm,price_fair_value_ttm,price_sales_ratio_ttm,price_to_book_ratio_ttm,price_to_free_cash_flows_ratio_ttm,price_to_operating_cash_flows_ratio_ttm,price_to_sales_ratio_ttm,quick_ratio_ttm,receivables_turnover_ttm,return_on_assets_ttm,return_on_capital_employed_ttm,return_on_equity_ttm,short_term_coverage_ratios_ttm,total_debt_to_capitalization_ttm) = 
    (EXCLUDED.symbol,EXCLUDED.asset_turnover_ttm,EXCLUDED.capital_expenditure_coverage_ratio_ttm,EXCLUDED.cash_conversion_cycle_ttm,EXCLUDED.cash_flow_coverage_ratios_ttm,EXCLUDED.cash_flow_to_debt_ratio_ttm,EXCLUDED.cash_per_share_ttm,EXCLUDED.cash_ratio_ttm,EXCLUDED.company_equity_multiplier_ttm,EXCLUDED.current_ratio_ttm,EXCLUDED.days_of_inventory_outstanding_ttm,EXCLUDED.days_of_payables_outstanding_ttm,EXCLUDED.days_of_sales_outstanding_ttm,EXCLUDED.debt_equity_ratio_ttm,EXCLUDED.debt_ratio_ttm,EXCLUDED.dividend_paid_and_capex_coverage_ratio_ttm,EXCLUDED.dividend_per_share_ttm,EXCLUDED.dividend_yiel_percentage_ttm,EXCLUDED.dividend_yiel_ttm,EXCLUDED.dividend_yield_ttm,EXCLUDED.ebit_per_revenue_ttm,EXCLUDED.ebt_per_ebit_ttm,EXCLUDED.effective_tax_rate_ttm,EXCLUDED.enterprise_value_multiple_ttm,EXCLUDED.fixed_asset_turnover_ttm,EXCLUDED.free_cash_flow_operating_cash_flow_ratio_ttm,EXCLUDED.free_cash_flow_per_share_ttm,EXCLUDED.gross_profit_margin_ttm,EXCLUDED.interest_coverage_ttm,EXCLUDED.inventory_turnover_ttm,EXCLUDED.long_term_debt_to_capitalization_ttm,EXCLUDED.net_income_per_ebtttm,EXCLUDED.net_profit_margin_ttm,EXCLUDED.operating_cash_flow_per_share_ttm,EXCLUDED.operating_cash_flow_sales_ratio_ttm,EXCLUDED.operating_cycle_ttm,EXCLUDED.operating_profit_margin_ttm,EXCLUDED.payables_turnover_ttm,EXCLUDED.payout_ratio_ttm,EXCLUDED.pe_ratio_ttm,EXCLUDED.peg_ratio_ttm,EXCLUDED.pretax_profit_margin_ttm,EXCLUDED.price_book_value_ratio_ttm,EXCLUDED.price_cash_flow_ratio_ttm,EXCLUDED.price_earnings_ratio_ttm,EXCLUDED.price_earnings_to_growth_ratio_ttm,EXCLUDED.price_fair_value_ttm,EXCLUDED.price_sales_ratio_ttm,EXCLUDED.price_to_book_ratio_ttm,EXCLUDED.price_to_free_cash_flows_ratio_ttm,EXCLUDED.price_to_operating_cash_flows_ratio_ttm,EXCLUDED.price_to_sales_ratio_ttm,EXCLUDED.quick_ratio_ttm,EXCLUDED.receivables_turnover_ttm,EXCLUDED.return_on_assets_ttm,EXCLUDED.return_on_capital_employed_ttm,EXCLUDED.return_on_equity_ttm,EXCLUDED.short_term_coverage_ratios_ttm,EXCLUDED.total_debt_to_capitalization_ttm);
    '''
    
    
    for symbol in required_companies_symbol:
        try:
            if (required_companies_symbol.index(symbol)%10000==0 and required_companies_symbol.index(symbol)!=0) or (required_companies_symbol.index(symbol)+1==len(required_companies_symbol)):
                print('Sending details to emails!')
                if (required_companies_symbol.index(symbol)+1==len(required_companies_symbol)):
                    message=f'Financial Ratios table has been updated! Total records in table are {len(required_companies_symbol)}'
                else:
                    message=f'{required_companies_symbol.index(symbol)} records have been updated in the Financial Ratio table.'
                send_mail(send_from='faghost6201@gmail.com',
                    subject='Financial Ratios Table Status',
                    text=message,
                    send_to=['bayo.billing@gmail.com','owaisahmed142002@gmail.com','alikhanhamza434@gmail.com','faghost6201@gmail.com'],
                    files=[])
            specific_stock=driver.execute_script('''
                var datas
                await fetch("https://fmpcloud.io/api/v3/ratios-ttm/'''+symbol+'''?apikey='''+api_key+'''", {
                "headers": {
                "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
                "accept-language": "en-GB,en-US;q=0.9,en;q=0.8",
                "cache-control": "no-cache",
                "pragma": "no-cache",
                "sec-fetch-dest": "document",
                "sec-fetch-mode": "navigate",
                "sec-fetch-site": "none",
                "sec-fetch-user": "?1",
                "upgrade-insecure-requests": "1"
              },
              "referrerPolicy": "strict-origin-when-cross-origin",
              "body": null,
              "method": "GET",
              "mode": "cors",
              "credentials": "include"
            }).then((response) => response.json()).then((data)=>datas=data)
            return datas ''')
            specific_stock_data=list(specific_stock[0].values())        
            hashed_key = hashlib.sha256(symbol.encode('utf-8')).hexdigest()
            specific_stock_data.insert(0,hashed_key)
            specific_stock_data.insert(1,symbol)
            specific_stock_data = replace_values(specific_stock_data, '', None)
            cur.execute(insert_sql,tuple(specific_stock_data))
            conn.commit()
        except Exception as e:
            print(e)
            conn=database_connection()
            cur = conn.cursor()
            cur.execute(insert_sql,tuple(specific_stock_data))
            conn.commit()
        


In [ ]:
if DB_ENV_PROD==1:    
    REMOTE_HOST = os.getenv('REMOTE_HOST')
    REMOTE_USERNAME = os.getenv('REMOTE_USERNAME')
    PKEY_PATH= os.environ.get('PKEY_PATH')
    
    conn_params = {
    'database': os.getenv('CLOUD_DB_NAME'), 
    'user': os.getenv('CLOUD_DB_USER'), 
    'password': os.getenv('CLOUD_DB_PASSWORD'), 
    'host': os.getenv('CLOUD_DB_HOST'), 
    'port': int(os.getenv('CLOUD_DB_PORT'))
    }
    def open_ssh_tunnel(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            tunnel = SSHTunnelForwarder((REMOTE_HOST),
                ssh_pkey=PKEY_PATH,
                ssh_username=REMOTE_USERNAME,
                remote_bind_address=(conn_params['host'],int(conn_params['port'])),
                )
            tunnel.start()
#             conn_params['port'] = tunnel.local_bind_port

            result = func(*args, **kwargs)

            tunnel.stop()
            return result
        return wrapper
    
#     @open_ssh_tunnel
    def query_make_table():
        conn = psycopg2.connect(**conn_params)
        cur = conn.cursor()
        cur.execute('''CREATE TABLE IF NOT EXISTS financial_ratios_ttm (
        hashed_key text NOT NULL PRIMARY KEY,
        symbol VARCHAR(255),
        asset_turnover_ttm float(24),
        capital_expenditure_coverage_ratio_ttm float(24),
        cash_conversion_cycle_ttm float(24),
        cash_flow_coverage_ratios_ttm float(24),
        cash_flow_to_debt_ratio_ttm float(24),
        cash_per_share_ttm float(24),
        cash_ratio_ttm float(24),
        company_equity_multiplier_ttm float(24),
        current_ratio_ttm float(24),
        days_of_inventory_outstanding_ttm float(24),
        days_of_payables_outstanding_ttm float(24),
        days_of_sales_outstanding_ttm float(24),
        debt_equity_ratio_ttm float(24),
        debt_ratio_ttm float(24),
        dividend_paid_and_capex_coverage_ratio_ttm float(24),
        dividend_per_share_ttm float(24),
        dividend_yiel_percentage_ttm float(24),
        dividend_yiel_ttm float(24),
        dividend_yield_ttm float(24),
        ebit_per_revenue_ttm float(24),
        ebt_per_ebit_ttm float(24),
        effective_tax_rate_ttm float(24),
        enterprise_value_multiple_ttm float(24),
        fixed_asset_turnover_ttm float(24),
        free_cash_flow_operating_cash_flow_ratio_ttm float(24),
        free_cash_flow_per_share_ttm float(24),
        gross_profit_margin_ttm float(24),
        interest_coverage_ttm float(24),
        inventory_turnover_ttm float(24),
        long_term_debt_to_capitalization_ttm float(24),
        net_income_per_ebtttm float(24),
        net_profit_margin_ttm float(24),
        operating_cash_flow_per_share_ttm float(24),
        operating_cash_flow_sales_ratio_ttm float(24),
        operating_cycle_ttm float(24),
        operating_profit_margin_ttm float(24),
        payables_turnover_ttm float(24),
        payout_ratio_ttm float(24),
        pe_ratio_ttm float(24),
        peg_ratio_ttm float(24),
        pretax_profit_margin_ttm float(24),
        price_book_value_ratio_ttm float(24),
        price_cash_flow_ratio_ttm float(24),
        price_earnings_ratio_ttm float(24),
        price_earnings_to_growth_ratio_ttm float(24),
        price_fair_value_ttm float(24),
        price_sales_ratio_ttm float(24),
        price_to_book_ratio_ttm float(24),
        price_to_free_cash_flows_ratio_ttm float(24),
        price_to_operating_cash_flows_ratio_ttm float(24),
        price_to_sales_ratio_ttm float(24),
        quick_ratio_ttm float(24),
        receivables_turnover_ttm float(24),
        return_on_assets_ttm float(24),
        return_on_capital_employed_ttm float(24),
        return_on_equity_ttm float(24),
        short_term_coverage_ratios_ttm float(24),
        total_debt_to_capitalization_ttm float(24));''')
        conn.commit()
        conn.close()
        print('Query Executed!')
        
    query_make_table()
                    
#     @open_ssh_tunnel
    def insert_burst_data(data):
        print('Started Inserting data into database')
        conn = psycopg2.connect(**conn_params)
        insert_sql = '''
        INSERT INTO financial_ratios_ttm (hashed_key,symbol,asset_turnover_ttm,capital_expenditure_coverage_ratio_ttm,cash_conversion_cycle_ttm,cash_flow_coverage_ratios_ttm,cash_flow_to_debt_ratio_ttm,cash_per_share_ttm,cash_ratio_ttm,company_equity_multiplier_ttm,current_ratio_ttm,days_of_inventory_outstanding_ttm,days_of_payables_outstanding_ttm,days_of_sales_outstanding_ttm,debt_equity_ratio_ttm,debt_ratio_ttm,dividend_paid_and_capex_coverage_ratio_ttm,dividend_per_share_ttm,dividend_yiel_percentage_ttm,dividend_yiel_ttm,dividend_yield_ttm,ebit_per_revenue_ttm,ebt_per_ebit_ttm,effective_tax_rate_ttm,enterprise_value_multiple_ttm,fixed_asset_turnover_ttm,free_cash_flow_operating_cash_flow_ratio_ttm,free_cash_flow_per_share_ttm,gross_profit_margin_ttm,interest_coverage_ttm,inventory_turnover_ttm,long_term_debt_to_capitalization_ttm,net_income_per_ebtttm,net_profit_margin_ttm,operating_cash_flow_per_share_ttm,operating_cash_flow_sales_ratio_ttm,operating_cycle_ttm,operating_profit_margin_ttm,payables_turnover_ttm,payout_ratio_ttm,pe_ratio_ttm,peg_ratio_ttm,pretax_profit_margin_ttm,price_book_value_ratio_ttm,price_cash_flow_ratio_ttm,price_earnings_ratio_ttm,price_earnings_to_growth_ratio_ttm,price_fair_value_ttm,price_sales_ratio_ttm,price_to_book_ratio_ttm,price_to_free_cash_flows_ratio_ttm,price_to_operating_cash_flows_ratio_ttm,price_to_sales_ratio_ttm,quick_ratio_ttm,receivables_turnover_ttm,return_on_assets_ttm,return_on_capital_employed_ttm,return_on_equity_ttm,short_term_coverage_ratios_ttm,total_debt_to_capitalization_ttm)
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        ON CONFLICT (hashed_key) DO UPDATE SET
        (symbol,asset_turnover_ttm,capital_expenditure_coverage_ratio_ttm,cash_conversion_cycle_ttm,cash_flow_coverage_ratios_ttm,cash_flow_to_debt_ratio_ttm,cash_per_share_ttm,cash_ratio_ttm,company_equity_multiplier_ttm,current_ratio_ttm,days_of_inventory_outstanding_ttm,days_of_payables_outstanding_ttm,days_of_sales_outstanding_ttm,debt_equity_ratio_ttm,debt_ratio_ttm,dividend_paid_and_capex_coverage_ratio_ttm,dividend_per_share_ttm,dividend_yiel_percentage_ttm,dividend_yiel_ttm,dividend_yield_ttm,ebit_per_revenue_ttm,ebt_per_ebit_ttm,effective_tax_rate_ttm,enterprise_value_multiple_ttm,fixed_asset_turnover_ttm,free_cash_flow_operating_cash_flow_ratio_ttm,free_cash_flow_per_share_ttm,gross_profit_margin_ttm,interest_coverage_ttm,inventory_turnover_ttm,long_term_debt_to_capitalization_ttm,net_income_per_ebtttm,net_profit_margin_ttm,operating_cash_flow_per_share_ttm,operating_cash_flow_sales_ratio_ttm,operating_cycle_ttm,operating_profit_margin_ttm,payables_turnover_ttm,payout_ratio_ttm,pe_ratio_ttm,peg_ratio_ttm,pretax_profit_margin_ttm,price_book_value_ratio_ttm,price_cash_flow_ratio_ttm,price_earnings_ratio_ttm,price_earnings_to_growth_ratio_ttm,price_fair_value_ttm,price_sales_ratio_ttm,price_to_book_ratio_ttm,price_to_free_cash_flows_ratio_ttm,price_to_operating_cash_flows_ratio_ttm,price_to_sales_ratio_ttm,quick_ratio_ttm,receivables_turnover_ttm,return_on_assets_ttm,return_on_capital_employed_ttm,return_on_equity_ttm,short_term_coverage_ratios_ttm,total_debt_to_capitalization_ttm) = 
        (EXCLUDED.symbol,EXCLUDED.asset_turnover_ttm,EXCLUDED.capital_expenditure_coverage_ratio_ttm,EXCLUDED.cash_conversion_cycle_ttm,EXCLUDED.cash_flow_coverage_ratios_ttm,EXCLUDED.cash_flow_to_debt_ratio_ttm,EXCLUDED.cash_per_share_ttm,EXCLUDED.cash_ratio_ttm,EXCLUDED.company_equity_multiplier_ttm,EXCLUDED.current_ratio_ttm,EXCLUDED.days_of_inventory_outstanding_ttm,EXCLUDED.days_of_payables_outstanding_ttm,EXCLUDED.days_of_sales_outstanding_ttm,EXCLUDED.debt_equity_ratio_ttm,EXCLUDED.debt_ratio_ttm,EXCLUDED.dividend_paid_and_capex_coverage_ratio_ttm,EXCLUDED.dividend_per_share_ttm,EXCLUDED.dividend_yiel_percentage_ttm,EXCLUDED.dividend_yiel_ttm,EXCLUDED.dividend_yield_ttm,EXCLUDED.ebit_per_revenue_ttm,EXCLUDED.ebt_per_ebit_ttm,EXCLUDED.effective_tax_rate_ttm,EXCLUDED.enterprise_value_multiple_ttm,EXCLUDED.fixed_asset_turnover_ttm,EXCLUDED.free_cash_flow_operating_cash_flow_ratio_ttm,EXCLUDED.free_cash_flow_per_share_ttm,EXCLUDED.gross_profit_margin_ttm,EXCLUDED.interest_coverage_ttm,EXCLUDED.inventory_turnover_ttm,EXCLUDED.long_term_debt_to_capitalization_ttm,EXCLUDED.net_income_per_ebtttm,EXCLUDED.net_profit_margin_ttm,EXCLUDED.operating_cash_flow_per_share_ttm,EXCLUDED.operating_cash_flow_sales_ratio_ttm,EXCLUDED.operating_cycle_ttm,EXCLUDED.operating_profit_margin_ttm,EXCLUDED.payables_turnover_ttm,EXCLUDED.payout_ratio_ttm,EXCLUDED.pe_ratio_ttm,EXCLUDED.peg_ratio_ttm,EXCLUDED.pretax_profit_margin_ttm,EXCLUDED.price_book_value_ratio_ttm,EXCLUDED.price_cash_flow_ratio_ttm,EXCLUDED.price_earnings_ratio_ttm,EXCLUDED.price_earnings_to_growth_ratio_ttm,EXCLUDED.price_fair_value_ttm,EXCLUDED.price_sales_ratio_ttm,EXCLUDED.price_to_book_ratio_ttm,EXCLUDED.price_to_free_cash_flows_ratio_ttm,EXCLUDED.price_to_operating_cash_flows_ratio_ttm,EXCLUDED.price_to_sales_ratio_ttm,EXCLUDED.quick_ratio_ttm,EXCLUDED.receivables_turnover_ttm,EXCLUDED.return_on_assets_ttm,EXCLUDED.return_on_capital_employed_ttm,EXCLUDED.return_on_equity_ttm,EXCLUDED.short_term_coverage_ratios_ttm,EXCLUDED.total_debt_to_capitalization_ttm);
        '''
        cur = conn.cursor()
        for record in data:
            if (data.index(record)%10000==0 and data.index(record)!=0) or (data.index(record)+1==len(data)):
                print('Sending details to emails!')
                if (data.index(record)+1==len(data)):
                    message=f'Financial Ratios table has been updated! Total records in table are {len(data)}'
                else:
                    message=f'{data.index(record)} records have been updated in the Financial Ratios table.'
                send_mail(send_from='faghost6201@gmail.com',
                    subject='Financial Ratios Table Status',
                    text=message,
                    send_to=['bayo.billing@gmail.com','owaisahmed142002@gmail.com','alikhanhamza434@gmail.com','faghost6201@gmail.com'],
                    files=[])
            try:
                cur.execute(insert_sql,tuple(record))
                conn.commit()
            except: 
                try:
                    conn = psycopg2.connect(**conn_params)
                except Exception as inside_e:
                    print(inside_e)
                    print('The index is:',data.index(record))
                cur = conn.cursor()
                cur.execute(insert_sql,tuple(record))
                conn.commit()
        conn.close()
        print('All the data has been inserted!')




    options = uc.ChromeOptions()
    options.headless=True
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')

    driver = uc.Chrome(executable_path=ChromeDriverManager().install(),use_subprocess=True,options=options)
    driver.get('https://fmpcloud.io/api/v3')
    driver.implicitly_wait(30)
    all_data=[]
    print('The data is being scraped!')
    for symbol in required_companies_symbol:
#         if len(all_data)==10:
#             break
        specific_stock=driver.execute_script('''
            var datas
            await fetch("https://fmpcloud.io/api/v3/ratios-ttm/'''+symbol+'''?apikey='''+api_key+'''", {
            "headers": {
            "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
            "accept-language": "en-GB,en-US;q=0.9,en;q=0.8",
            "cache-control": "no-cache",
            "pragma": "no-cache",
            "sec-fetch-dest": "document",
            "sec-fetch-mode": "navigate",
            "sec-fetch-site": "none",
            "sec-fetch-user": "?1",
            "upgrade-insecure-requests": "1"
          },
          "referrerPolicy": "strict-origin-when-cross-origin",
          "body": null,
          "method": "GET",
          "mode": "cors",
          "credentials": "include"
        }).then((response) => response.json()).then((data)=>datas=data)
        return datas ''')
        specific_stock_data=list(specific_stock[0].values())        
        hashed_key = hashlib.sha256(symbol.encode('utf-8')).hexdigest()
        specific_stock_data.insert(0,hashed_key)
        specific_stock_data.insert(1,symbol)
        specific_stock_data = replace_values(specific_stock_data, '', None)
        all_data.append(specific_stock_data)
                    
    insert_burst_data(all_data)
    print('Data has been inserted successfully!')





